# Ensuring that the Mesh Poses saved can be utilized!

In [15]:
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import os

%matplotlib inline

## Goals
- Mesh is captured 30 seconds w/ camera pose
- Find a way to reconstruct what meshes have been captured (full 3d reconstruct w/ MeshLab)
- Find correlation between mesh poses and camera poses
- Perform unprojection to test if it works

In [23]:
# PATH = '/home/andang/workspace/CV_Lab/Hololens-Project/datasets/john_place/raw-05-09-18-46/'
PATH = '/home/andang/workspace/Computational_Vision/HL2/Hololens-Project/datasets/an_place/vid1/'
PATH_MESH = PATH + 'mesh/'
# PATH_MESH = PATH + 'gt_mesh/'

In [36]:
mesh_hmap = {}

for path in os.listdir(PATH_MESH):
    split_names = (path.split('.txt')[0]).split('_')
    info_type = split_names[1]
    timestamp = split_names[2]
    index = split_names[3]
    data = np.loadtxt(PATH_MESH+path) #load up info

    if not (timestamp in mesh_hmap):
        mesh_hmap[timestamp] = {}

    if not index in mesh_hmap[timestamp]:
        mesh_hmap[timestamp][index] = {}


    mesh_hmap[timestamp][index][info_type] = data

In [40]:
#open3d rendering
open3d_meshes = []

length = 0
mu = np.zeros((4,1))
var = np.zeros((4,1))
for timestamp,j in mesh_hmap.items():
    for i,j in mesh_hmap[timestamp].items():
        vertex_positions = j['vertex'].copy()
        triangle_indices = j['triangles'].copy()
        pose = j['pose'].copy()
        vertex_position_scale = j['vertexscale'].copy()
        vertex_normals = j['normals'].copy()

        vertex_positions[:, 0:3] = vertex_positions[:, 0:3] * vertex_position_scale
        # vertex_positions = (vertex_positions/vertex_positions[:,3:]) @ pose
        vertex_positions = (vertex_positions) @ pose
        
        mu += vertex_positions.sum(axis=0).reshape(4,1)
        length += vertex_positions.shape[0]
        
        d = np.linalg.norm(vertex_normals, axis=1)
        vertex_normals[d > 0, :] = vertex_normals[d > 0, :] / d[d > 0, np.newaxis]
        vertex_normals = vertex_normals @ pose

        open3d_mesh = o3d.geometry.TriangleMesh()
        open3d_mesh.vertices = o3d.utility.Vector3dVector(vertex_positions[:,0:3])
        open3d_mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_normals[:, 0:3])
        open3d_mesh.triangles = o3d.utility.Vector3iVector(triangle_indices)
        open3d_meshes.append(open3d_mesh)
        
        print(vertex_positions[0])
    break
    
print(mu/length)

o3d.visualization.draw_geometries(open3d_meshes, mesh_show_back_face=True)

[0.71724674 0.66985475 1.67591114 1.        ]
[0.17895951 0.67083314 1.62824422 1.        ]
[-1.09625411  0.64648209 -0.5195643   1.        ]
[ 1.50353629  0.64413658 -1.30113722  1.        ]
[[ 0.40587779]
 [-0.34135125]
 [-0.02955146]
 [ 1.        ]]


In [41]:
full_mesh = o3d.geometry.TriangleMesh()
for open3d_mesh in open3d_meshes:
    full_mesh += open3d_mesh

In [48]:
np.min(np.asarray(full_mesh.vertices).T,axis=1), np.max(np.asarray(full_mesh.vertices).T,axis=1)

(array([-1.28719793, -1.00830966, -2.0430628 ]),
 array([2.43657027, 0.95291832, 1.70193608]))

In [35]:
o3d.io.write_triangle_mesh('test_stuff.obj',full_mesh,write_ascii=True,write_triangle_uvs=True, write_vertex_colors=True, write_vertex_normals=True)

True

## SUCCESS! Open3D can open a 3d reconstruction mesh now